In [1]:
%pip install azure-ai-inference[opentelemetry]
%pip install azure-search-documents 
%pip install azure-identity
%pip install openai

zsh:1: no matches found: azure-ai-inference[opentelemetry]
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import dotenv
from azure.search.documents.models import VectorizedQuery, VectorizableTextQuery

dotenv.load_dotenv()

# Leer las variables definidas en el archivo .env

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")

AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME="text-embedding-ada-002"

AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
AZURE_SEARCH_ADMIN_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")

openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX,
    credential=AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
)

def get_embedding(text):
    return openai_client.embeddings.create(
        model=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
        input=text
    ).data[0].embedding

user_question = "Nitrogen fertilization is important for?"
user_question_vector = get_embedding(user_question)
print(user_question_vector)

[0.004052820615470409, -0.025624703615903854, -0.004026923794299364, -0.00662305997684598, 0.002162367571145296, 0.024006163701415062, -0.03247435763478279, 0.005130767356604338, -0.03224128857254982, -0.01618538610637188, 0.011595210060477257, 0.008655943907797337, -0.008261021226644516, 0.02783886343240738, 0.002270809607580304, 0.005101633723825216, 0.037083957344293594, -0.012106668204069138, 0.01625012792646885, -0.009581748396158218, -0.01714356057345867, -0.002261098474264145, 0.012203780934214592, 0.0065486072562634945, 0.0030331413727253675, 0.014489157125353813, 0.02104100212454796, -0.026544032618403435, 0.0013498611515387893, -0.005134004168212414, 0.031101837754249573, 0.009685334749519825, -0.020328843966126442, -0.004913883283734322, -0.013045420870184898, -0.02219340018928051, 0.005752285942435265, -0.0011523994617164135, 0.023967320099473, -0.005208457354456186, 0.023811940103769302, -0.00525377644225955, 0.014683381654322147, -0.01390648353844881, -0.01298067905008792

In [4]:
search_results = search_client.search(
    None,
    top=3,
    vector_queries=[
        VectorizableTextQuery( 
            text=user_question, k_nearest_neighbors=3, fields="text_vector"
        )
    ],
)

for result in search_results:
    print("Chunk ID:", result["chunk_id"])
    print("Title:", result["title"])
    print("Text:", result["chunk"])
    print()

Chunk ID: f853c78a3612_aHR0cHM6Ly9zdGd0b3BpY29zcmFnLmJsb2IuY29yZS53aW5kb3dzLm5ldC9yYWctZG9jcy9waW5oZXJvMjAxNi5wZGY1_pages_6
Title: pinhero2016.pdf
Text: nitrogen fertility increases yield but usually decreases storage quality 
(Booth and Shaw, 1981). High or excessive nitrogen stimulates overly large plant canopies, may 
delay maturity of the potato crop, lowers the dry matter of the tubers, and results in poorer skin 
set. Nitrogen (N) fertility can have a role because of its influence on stimulating tuber growth 
and size. Nitrogen applied late in the season delays maturity, reduces specific gravity, and 
increases the reducing sugar content. Nitrogen should be applied in amounts that do not exceed 
the crop’s needs, and it should not be applied within 4–6 weeks of vine killing (Knowles and 
Plissey, 2008). Some studies have shown that N applied at tuber initiation can increase the 
incidence of hollow heart and brown center (Hiller and Thornton, 1993). Other studies have 
shown that

In [6]:
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME=["gpt-4o-mini", "grok-3", "DeepSeek-R1", "gpt-4o"][0]

context = ""
for result in search_results:
    context += result["chunk"] + "\n\n"

SYSTEM_MESSAGE = f"""
You are an AI Assistant.
Be brief in your answers. Answer ONLY with the facts listed in the retrieved text.

Context:
{context}
"""

USER_MESSAGE = user_question

response = openai_client.chat.completions.create(
    model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

answer = response.choices[0].message.content
print(answer)

Nitrogen fertilization is important for enhancing plant growth, improving crop yield, and increasing soil fertility.
